# AWS DeepRacer Model Evaluator with Claude LLM

> _This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio_

This hands-on workshop demonstrates how to build a conversational agent using Amazon Bedrock with Anthropic Claude, a large language model (LLM), combined with the Langchain library. The agent is designed to provide insights and recommendations about AWS DeepRacer models and training.

The workshop shows how to:

* Create custom Langchain tools to allow the agent to interface with the AWS DeepRacer  service API. This includes listing available models, downloading model artifacts, and extracting model metadata like the training data and reward function.

* Initialize a ReAct agent in Langchain and make the custom tools available to it. The agent can reason about which tools to invoke based on the user's questions.

* Use prompting techniques like few-shot learning to improve the agent's reasoning capabilities with just a few examples.

* Handle errors gracefully if the agent's responses don't match the expected format.

* Leverage the custom tools to enable the agent to provide insights about an AWS DeepRacer model's training data, hyperparameters, reward function and more.

By the end of the hands-on workshop, attendees will be able to build conversational agents using LLMs that can integrate with AWS services via custom interfaces. The key takeaways are being able to extend an agent's capabilities using tools, architect a modular agent, and applying prompting techniques to improve reasoning.


## Environment setup

Before running the rest of this notebook, you'll need to install the necessary libraries and ensure there is a working connection to Amazon Bedrock and the AWS DeepRacer service API.

For this lab, we will need some additional dependencies: 
- Boto3, Python SDK for AWS
- ChromaDB, to store vector embeddings
- Langchain,  framework for developing applications powered by language models
- PyYAML, YAML parser

⚠️ You will see pip dependency errors, you can safely ignore these errors. ⚠️

IGNORE ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behavior is the source of the following dependency conflicts.

In [4]:
# %pip install --no-build-isolation --quiet \
#     "boto3==1.28.62" \
#     "chromadb==0.4.13" \
#     "langchain==0.0.325"  \
#     "pyyaml==6.0.1"

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.32.84 requires botocore==1.34.84, but you have botocore 1.31.85 which is incompatible.
awscli 1.32.84 requires s3transfer<0.11.0,>=0.10.0, but you have s3transfer 0.7.0 which is incompatible.
sagemaker 2.215.0 requires boto3<2.0,>=1.33.3, but you have boto3 1.28.62 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [9]:
%pip install --no-build-isolation --quiet \
    "boto3>=1.33.3,<2.0" \
    "chromadb==0.4.13" \
    "langchain==0.0.325"  \
    "pyyaml==6.0.1" \
    "botocore==1.34.84" \
    "s3transfer>=0.10.0,<0.11.0"

Note: you may need to restart the kernel to use updated packages.


### Add util libraries

The util libraries contain code for interacting with the AWS DeepRacer service API, extract relevant information from DeepRacer models etc.

In [2]:
import os
import sys

module_path = "./utils"
sys.path.append(os.path.abspath(module_path))

from utils import print_ww, deepracer, deepracer_model, s3, cloudformation

This notebook also need access to AWS infrastructure which has been deployed into the used AWS account as part of the workshop setup. The setup of AWS infrastructure was done with Cloudformation.

In [3]:
#If using your own AWS account please update with the name of the Cloudformation stack used to deploy the AWS infrastructure
CLOUDFORMATION_STACK_NAME="sgdomainanduser"

# DEEPRACER_EXPORT_S3_BUCKET: S3 Bucket used for copying S3 models to from the DeepRacer service.
# DEEPRACER_COPY_TO_S3_IAM_ROLE_ARN: IAM role which is passed to the DeepRacer service when copying models to the S3 bucket
DEEPRACER_EXPORT_S3_BUCKET, DEEPRACER_COPY_TO_S3_IAM_ROLE_ARN = cloudformation.get_stack_outputs(CLOUDFORMATION_STACK_NAME)
print(f"DEEPRACER_EXPORT_S3_BUCKET = {DEEPRACER_EXPORT_S3_BUCKET}")
print(f"DEEPRACER_COPY_TO_S3_IAM_ROLE_ARN = {DEEPRACER_COPY_TO_S3_IAM_ROLE_ARN}")

DEEPRACER_EXPORT_S3_BUCKET = sgdomainanduser-deepracermodelexportbucket-7mpelef4xgqf
DEEPRACER_COPY_TO_S3_IAM_ROLE_ARN = arn:aws:iam::577638359963:role/sgdomainanduser-DeepRacerCopyToS3Role-aQ62UkdSxRn8


### DeepRacer service connection

**Note:** The AWS DeepRacer service is only available in AWS region us-east-1. 

#### Verify the connection to DeepRacer

Let´s check the connection to the service by listing available AWS DeepRacer models. 

First enter your racer name, replacing the example 'superfast_racer' below.  This can be found in 'Your racer profile' section of the DeepRacer Console

In [4]:
RACER_PROFILE_NAME = "superfast-racer"

In [5]:
models = deepracer.list_models()

for model in models:
    if model['CreatedByAlias']==RACER_PROFILE_NAME: 
        print(f"Name: {model['ModelName']}")


Name: tr-reinvent-v1
Name: a20240816
Name: rr20240728c-1


This call will return the names of the DeepRacer models in your account, to have a conversation with the agent about the models they should have either been created in the console or imported and then evaluated in the console.

### Bedrock service connection

In [8]:
import boto3
bedrock_client = boto3.client('bedrock-runtime')

### LLM setup

In this lab we will use the following LLM:s
- Claude instant v1, for the reasoning and act (ReAct) agent as well as for building a DeepRacer Question & Answer LLM tool. 
- Amazon Titan embeddings model, to allow the DeepRacer Question & Answer LLM tool to do semantic search on the DeepRacer documentation to improve the response quality

In [10]:
from langchain.llms.bedrock import Bedrock
from langchain.embeddings import BedrockEmbeddings

llm_claude_instant = Bedrock(
    model_id="anthropic.claude-instant-v1",
    client=bedrock_client,
    model_kwargs={ "temperature": 0, "max_tokens_to_sample": 3000},
)

bedrock_embeddings_client = BedrockEmbeddings(
        client=bedrock_client,
        model_id="amazon.titan-embed-text-v1"
    )

## Provide more upto date AWS DeepRacer knowledge to the LLM

Claude instant has previous knowledge of AWS DeepRacer but the knowledge is not up to date with the latest version of the AWS DeepRacer developer guidelines. To make the model aware of more recent additions we can create an Langchain chain which utilize Retrieval Augmentation Generation (RAG). 

We will not cover how RAG works in detail in this workshop, for more information:
- [Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks](https://arxiv.org/abs/2005.11401)
- [Bedrock workshop](https://github.com/aws-samples/amazon-bedrock-workshop/tree/main/03_QuestionAnswering)


### AWS DeepRacer knowledge base tool

This tool is another Langchain chain specifically designed to answer AWS DeepRacer related questions. This chain use Retrieval Augmentation Generation (RAG) to get documents relevant to the asked question to provide a better foundation to answer AWS DeepRacer related questions.

The AWS DeepRacer developer guideline was loaded and split into chunks and converted to embeddings.

![](./images/Embeddings_lang.png)

Embeddings are vector representations of words that encode semantic meaning. In retrieval augmented generation, embeddings help find relevant external knowledge to augment the capabilities of a generative chatbot LLM. The chatbot LLM uses the retrieved embeddings during generation to construct more knowledgeable and relevant response.

![](./images/Chatbot_lang.png)

#### Setup the vector store

In [11]:
from langchain.vectorstores import Chroma

# Load the pre-made embeddings into the Vector store.
vectorstore_chromadb = Chroma(persist_directory="./persistent/chroma_db", embedding_function=bedrock_embeddings_client)

#### Create the AWS DeepRacer QA chain

For building the RAG chain we utilize one of the Langchain chains, [Retrieval QA chain](https://js.langchain.com/docs/modules/chains/popular/vector_db_qa/), which has built in support for RAG.

The prompt is using a couple of techniques to ensure AWS DeepRacer relevant responses:
- The documents retrieved from the vector store is inserted into the {context} variable to make them available to the LLM
- Reduce hallucinations by requiring the LLM to use the information retrieved via RAG to answer the question

In [12]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

prompt_template = """Human: Given an AWS DeepRacer related question, provide a detailed answer.

<context>
{context}
</context>

Here is the human's next reply:
<human_reply>
 {question}
</human_reply>

If the answer cannot be found in the context, the AI truthfully says it does not know the answer.

Assistant:"""

prompt = PromptTemplate.from_template(prompt_template)

deepracer_qa_chain = RetrievalQA.from_chain_type(
    llm=llm_claude_instant,
    chain_type="stuff",  # add all returned documents
    retriever=vectorstore_chromadb.as_retriever(
        search_type="similarity",
        search_kwargs={
            "k": 3
        },  # return the top 3 similar documents from the vector store
    ),
    # return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
)


#### Create a Langchain custom tool

Now we package the AWS DeepRacer QA chain as Langchain custom tool and make it available to the agent

In [13]:
from langchain.agents import Tool
from pydantic import BaseModel, Field

# Initialize the LLM tool
deepracer_knowledge_tool = Tool.from_function(
    name='AWS DeepRacer knowledge base',
    func=deepracer_qa_chain.run,
    description='Useful when you need to answer generic questions about AWS DeepRacer, Input: the question to answer, Output: Answer to the question',
)

In [14]:
tools = [
        deepracer_knowledge_tool,
]

## Using ReAct: Synergizing Reasoning and Acting in Language Models Framework
Large language models can generate both explanations for their reasoning and task-specific responses in an alternating fashion.

Producing reasoning explanations enables the model to infer, monitor, and revise action plans, and even handle unexpected scenarios. The action step allows the model to interface with and obtain information from external sources such as knowledge bases or environments.

Here we will initialize the chain responsible for deciding what step to take next. This is powered by a language model and a prompt. The inputs to this chain are:

- List of available tools
- User input
- Any previously executed steps (intermediate_steps)

This chain then returns either the next action to take or the final response to send to the user (AgentAction or AgentFinish).

Different [Langchain agent types](https://python.langchain.com/docs/modules/agents/agent_types/) have different prompting styles for reasoning, different ways of encoding input, and different ways of parsing the output.


In [15]:
from langchain.agents import initialize_agent, AgentType

react_agent = initialize_agent(tools,
                               llm=llm_claude_instant,
                               agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                               verbose=True, # displays the intermediate steps the agent performs
                               return_intermediate_steps=True,
                               #handle_parsing_errors=True,
                               )

Let´s have a look at the default prompt used for this chain.

Please note that the prompt template has the following structure:
```

Answer the following questions as best you can. You have access to the following tools:

{available tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of {tool names}
Action Input: the input to the action\nObservation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought: {agent_scratchpad}

```

The default prompt instructs the model how to reason and interact with the available tools.

In [16]:
react_agent.agent.llm_chain.prompt.template

'Answer the following questions as best you can. You have access to the following tools:\n\nAWS DeepRacer knowledge base: Useful when you need to answer generic questions about AWS DeepRacer, Input: the question to answer, Output: Answer to the question\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [AWS DeepRacer knowledge base]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}'

Once the agent ```react_agent``` is setup we can invoke the agent by passing a prompt/question as seen blow. Feel free to adjust the prompt/question on subsequent calls.

When running the below command you should see how the agent starts to reason what it need to do in order to answer the question, what tools is uses and their input. 
If all goes well the agent should produce a final answer.

The question below the agent usually manage to resolve in one plan->act->observe loop and you should see a "> Finished chain." with the final answer.


⚠️ **If you get an access denied issue when running the below cell, please check in Amazon Bedrock that both, Claude Instant and Titan Embeddings G1 – Text, has a status of Access granted.** ⚠️

⚠️ **If you get an OutputParserException. This can happen sometimes due to that the LLM do not return the response to the question in the correct format. This we will improve later on in the lab by introducing a few-shot prompt, provide some examples in the prompt on how the LLM should respond** ⚠️

In [ ]:
question = "What is AWS DeepRacer?"
print_ww(react_agent(question)["output"])

This question showcase one issue that can happen when using agents. When resolving this question the agent usually ends up in a loop with the same thought and action each time until it can´t follow the format it has been instructed to.

This we will improve later on in the lab by introducing a few-shot prompt, provide some examples in the prompt on how the LLM should reason, and have the agent manage parsing errors.

In [18]:
try:
    question = "How do I get started with training a DeepRacer model? anything specific I should think of?"
    print_ww(react_agent(question)["output"])
except Exception as e:
    print(f"\n\nThe agent encountered an error! \n {e}")



> Entering new AgentExecutor chain...
 Here is my attempt at answering the question using the provided format:

Question: How do I get started with training a DeepRacer model? anything specific I should think of?

Thought: I should check the AWS DeepRacer knowledge base to see the recommended steps for getting started with training a model.

Action: AWS DeepRacer knowledge base
Action Input: steps for getting started with training a DeepRacer model

Observation:  Based on the provided context, here are the key steps for training an AWS DeepRacer model:

1. Create a model by specifying a name and selecting a simulation track environment. 

2. Create your reward function to define success and failure for the agent.

3. Explore the action space to understand how actions like steering and throttle impact movement. 

4. Tune hyperparameters like learning rate and discount factor to optimize training.

5. Monitor the training job progress to see how the model is learning over time. 

6. Tr

## Add more tools to improve the LLMs capability to analyze DeepRacer models

So far the LLM can only answer generic information about AWS DeepRacer which is part of the [AWS DeepRacer developer guide](https://docs.aws.amazon.com/deepracer/latest/developerguide/what-is-deepracer.html). 
In this section we will integrate the LLM with the AWS DeepRacer service API so the LLM can interact with different models you have access to in this AWS account.

### Create a tool to list AWS DeepRacer models

When invoked by the agent this tool will list available DeepRacer models located in the AWS DeepRacer service. It has been implemented as a Langchain custom tool and use custom code to call the DeepRacer service API when invoked by the agent

In [19]:
from langchain.tools import BaseTool


class DeepRacerListModelsTool(BaseTool):
    name = "List AWS DeepRacer models"
    description = """
    Use this tool when you need to list all AWS DeepRacer models. Contains information on who created the model, model name, description.
    """

    def _run(self, input=None):
        # List all available DeepRacer models
        models = deepracer.list_models()

        # Extract only relevant attributes from the model objects to make it easier for the ReAct LLM to answer our questions
        models_found = []
        for model in models:
            if model['CreatedByAlias']==RACER_PROFILE_NAME: 
                models_found.append(
                    {
                        key: model[key]
                        for key in model.keys()
                        & {
                            "ModelName",
                            "ModelDescription",
                            "CreatedByAlias",
                            "CarConfiguration",
                        }
                    }
                )
        return models_found

    def _arun(self, radius: int):
        raise NotImplementedError("This tool does not support async")

### Create a tool to get AWS DeepRacer model details.

Downloads an AWS DeepRacer model and extract relevant parts to allow the LLM to do an analysis of its training and performance.

In [20]:
import os
from pathlib import Path

from langchain.tools import BaseTool
from pydantic import BaseModel, Field


class ModelAnalysisInput(BaseModel):
    model_name: str = Field()


class DeepRacerModelAnalysisTool(BaseTool):
    name = "AWS DeepRacer model details"
    description = """Use this tool to get detailed information about an AWS DeepRacer model. Input: Expects the model_name as string input. Output: Python dictionary"""
    args_schema = ModelAnalysisInput

    def _run(self, model_name=None):
        try:
            # Copy the DeepRacer model from AWS DeepRacer service to a S3 bucket.
            formatted_model_name = model_name.strip()
            target_s3_bucket = DEEPRACER_EXPORT_S3_BUCKET
            model_s3_prefix = deepracer.copy_model_to_s3_if_model_does_not_exist(
                formatted_model_name,
                target_s3_bucket,
                DEEPRACER_COPY_TO_S3_IAM_ROLE_ARN,
            )

            # Extract relevant information from the downloaded model files.
            model = deepracer_model.DeepRacerModel(target_s3_bucket, model_s3_prefix)
            model_data = {}
            model_data["model_metadata_used_for_training"] = model.get_model_meta_data()
            model_data[
                "reward_function_used_for_training"
            ] = model.get_reward_function()
            model_data[
                "hyper_parameters_used_for_training"
            ] = model.get_hyper_parameters()
            model_data["evaluation_results"] = model.get_evaluation_metrics()
            model_data["training_results"] = model.get_training_metrics()
            model_data["track_meta_data"] = model.get_track_meta_data()
            return model_data
        except FileNotFoundError as e:
            return f"Model with name {model_name} does not exist, {e}"

    def _arun(self, radius: int):
        raise NotImplementedError("This tool does not support async")


/opt/conda/lib/python3.10/site-packages/pydantic/_internal/_fields.py:160: UserWarning: Field "model_name" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


After having created our third and final tool, we add the 3 tools to a list that will be passed, as a positional argument, to the agent initialization function. Making it possible for the agent to use them when resolving prompts.

In [21]:
tools = [
        DeepRacerModelAnalysisTool(),
        DeepRacerListModelsTool(),
        deepracer_knowledge_tool,
]

In [22]:
from langchain.agents import initialize_agent, AgentType

react_agent = initialize_agent(tools,
                               llm=llm_claude_instant,
                               agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                               verbose=True, # verbose logs is turned on to give insights into how the agent will reason and use the tools
                               return_intermediate_steps=True,
                               handle_parsing_errors=True,
                               )

Before testing out the newly added tools let´s also improve the agents ability to reason, observe and act in the required format.

One way to do this is to introduce a few-shot prompt, eg: provide a couple of reasoning examples.
In the below prompt we have also adapted the default prompt to include ```/n/nHuman``` and ```/n/nAssistant```. Claude is trained to fill in text for the Assistant role as part of an ongoing dialogue between a human user (Human:) and an AI assistant (Assistant:).

For more details on prompting with Anthropic Claude see the [Anthropic prompting guide](https://docs.anthropic.com/claude/docs/introduction-to-prompt-design)

Another way is to enable ```handle_parsing_errors=True``` as we have done in the above cells. Doing this the agent will try to remediate the error by it self.

```
Thought: This gives a good overview but does not provide all the necessary details. I should check the model details next.

Action: AWS DeepRacer model details

Observation: Invalid Format: Missing 'Action Input:' after 'Action:'
Thought: Checking the model details did not provide the expected response. Let me try listing the available models to get more context.

```


In [23]:
import json

prompt_template = """Human: Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do, Also try to follow steps mentioned above.
Action: the action to take, should be one of {tools}.
Action Input: the input to the action.
Observation: the result of the action.
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer.
Final Answer: the final answer to the original input question.

<examples>
<example>
Question: What is AWS DeepRacer?
Thought: I need to get more information on what AWS DeepRacer is, I should check the AWS DeepRacer knowledge base
Action: AWS DeepRacer knowledge base
Action Input: What is AWS DeepRacer?
Observation:  AWS DeepRacer is the fastest way to get rolling with reinforcement learning (RL), literally, with a fully autonomous 1/18th scale race car driven by reinforcement learning, 3D racing simulator, and a global racing league.
Final Answer: AWS DeepRacer is the fastest way to get rolling with reinforcement learning (RL), literally, with a fully autonomous 1/18th scale race car driven by reinforcement learning, 3D racing simulator, and a global racing league.
</example>
<example>
Question: how can I improve my-deepracer-model?
Thought: I need to check the details and performance of the AWS DeepRacer model to answer this question.
Action: AWS DeepRacer model details
Action Input: my-deepracer-model
Observation: ....
Thought: I should check the AWS DeepRacer knowledge base on how to improve a DeepRacer model
Action: AWS DeepRacer knowledge base
Action Input: Improve deepracer model
Observation: ....
Thought: With the details about the model and knowledge on how to improve a model I know have enough information.
Final Answer: The my-deepracer-model model is designed and trained to follow the center line of the track. The model details, reward function, and hyperparameters used for training provide evidence that this model prioritizes staying on the center line, making it a good performing model for that task.
</example>
</examples>

You are an AWS DeepRacer Q&A bot, give a detailed answer to the question.
You can only answer question not perform any changes to any DeepRacer models.
Question: {input}


Assistant:
{agent_scratchpad}"""

# Create a prompt template and inject the names of all available tools in the {tools} variable. This can be done using partial_variables
tool_names = json.dumps([d.name for d in tools])
template = PromptTemplate(
    template=prompt_template,
    input_variables=["input", "agent_scratchpad"],
    partial_variables={"tools": tool_names},
)
react_agent.agent.llm_chain.prompt = template


With the addition of DeepRacerListModelsTool and DeepRacerModelAnalysisTool custom tools the agent is now able to:
1. List available AWS DeepRacer models
2. Download a named DeepRacer model and extract: 
     - training data
     - evaluation data
     - extract model meta data, like action space, hyperparameters etc
     - extract the reward function

As we say earlier in the lab the agent has access to some DeepRacer models in this AWS account.

In [24]:
for model in models:
    if model['CreatedByAlias']==RACER_PROFILE_NAME: 
        print(f"Name: {model['ModelName']}")

Name: tr-reinvent-v1
Name: a20240816
Name: rr20240728c-1


These DeepRacer model names can be used when asking questions to the agent. 

In this question the agent will use the **List AWS DeepRacer models** tool to via the DeepRacer service API get the names of the DeepRacer models in the current AWS account. After the **Finished chain** is the output that is returned to the user. 

The text that is seen before the **Finished chain** is just logs produced due to that we have **verbose=True** turned on for the LLM used by the agent. This is to better highlight how the agent is reasoning and interact with the tools.

In [36]:
question = "What are the names of DeepRacer models I have access to?"
print_ww(react_agent(question)["output"])



> Entering new AgentExecutor chain...
 Here is the response in the specified format:

Question: What are the names of DeepRacer models I have access to?
Thought: To answer this question, I need to know the names of the DeepRacer models available.
Action: List AWS DeepRacer models
Action Input: 
Observation: [{'CreatedByAlias': 'superfast-racer', 'CarConfiguration': {'NeuralNetwork': 'DEEP_CONVOLUTIONAL_NETWORK_SHALLOW', 'Sensors': ['FRONT_FACING_CAMERA']}, 'ModelName': 'tr-reinvent-v1-clone'}, {'CreatedByAlias': 'superfast-racer', 'CarConfiguration': {'NeuralNetwork': 'DEEP_CONVOLUTIONAL_NETWORK_SHALLOW', 'Sensors': ['FRONT_FACING_CAMERA']}, 'ModelName': 'tr-reinvent-v1'}, {'CreatedByAlias': 'superfast-racer', 'ModelDescription': 'rr20240728c-1-clone for reinvent track', 'ModelName': 'a20240816', 'CarConfiguration': {'NeuralNetwork': 'DEEP_CONVOLUTIONAL_NETWORK_SHALLOW', 'Sensors': ['FRONT_FACING_CAMERA']}}, {'CreatedByAlias': 'superfast-racer', 'CarConfiguration': {'NeuralNetwork': 

In this question the agent will use the **AWS DeepRacer model details** tool to via the DeepRacer service API extract **MODEL_NAME** model details (hyperparameters, reward function, training metrics, evaluation metrics and track used). This data is then returned to the agent as json and the agent will search for which track was used for the evaluation

After the **Finished chain** is the output that is returned to the user. 

Change the MODEL_NAME variable to the name of the model you want to learn about

In [37]:
MODEL_NAME="tr-reinvent-v1-clone"
COMPARE_MODEL_NAME="AtoZ-CCW-Steering-Penalty"

In [43]:
question = "Which track was " + MODEL_NAME + " trained and evaluated on?"
print_ww(react_agent( question)["output"])



> Entering new AgentExecutor chain...
 Here is the detailed response:

Question: Which track was tr-reinvent-v1-clone trained and evaluated on?

Thought: To answer this question, I need to check the details of the tr-reinvent-v1-clone model.

Action: AWS DeepRacer model details
Action Input: tr-reinvent-v1-clone
Could not obtain evaluation metrics 'Contents'

Observation: {'model_metadata_used_for_training': {'action_space_type': 'discrete', 'sensor': ['FRONT_FACING_CAMERA'], 'action_space': [{'steering_angle': -30, 'speed': 1.2, 'index': 0}, {'steering_angle': -30, 'speed': 1.4, 'index': 1}, {'steering_angle': -30, 'speed': 1.6, 'index': 2}, {'steering_angle': -20, 'speed': 1.8, 'index': 3}, {'steering_angle': -20, 'speed': 2, 'index': 4}, {'steering_angle': -20, 'speed': 2.2, 'index': 5}, {'steering_angle': -15, 'speed': 2.4, 'index': 6}, {'steering_angle': -15, 'speed': 2.6, 'index': 7}, {'steering_angle': -15, 'speed': 2.8, 'index': 8}, {'steering_angle': -10, 'speed': 3, 'index'

Let´s try some more questions to see how the agent reason and use the tools. 

In [25]:
question = "Is " + MODEL_NAME + " or " + COMPARE_MODEL_NAME + " the better model? please also state why"
print_ww(react_agent( question)["output"])



> Entering new AgentExecutor chain...
 Here is the detailed response:

Thought: To determine which model is better, I need details on the models' performance and training methodology.

Action: AWS DeepRacer model details
Action Input: rr20240728c-1Could not obtain evaluation metrics 'Contents'
Could not obtain training metrics 'Contents'

Observation: {'model_metadata_used_for_training': {'sensor': ['FRONT_FACING_CAMERA'], 'action_space_type': 'discrete', 'action_space': [{'steering_angle': -30, 'speed': 1.4, 'index': 0}, {'steering_angle': -30, 'speed': 1.2, 'index': 1}, {'steering_angle': -20, 'speed': 1.9, 'index': 2}, {'steering_angle': -10, 'speed': 2.5, 'index': 3}, {'steering_angle': -10, 'speed': 2.7, 'index': 4}, {'steering_angle': 0, 'speed': 3, 'index': 5}, {'steering_angle': 0, 'speed': 3.3, 'index': 6}, {'steering_angle': 0, 'speed': 3.5, 'index': 7}, {'steering_angle': 0, 'speed': 4, 'index': 8}, {'steering_angle': 10, 'speed': 2.9, 'index': 9}, {'steering_angle': 10, '

In [44]:
question = "How can I improve the training for " + MODEL_NAME + "?"
print_ww(react_agent( question)["output"])



> Entering new AgentExecutor chain...
 Here is the detailed response:

Thought: I need more context on the tr-reinvent-v1-clone model to understand how it can be improved.

Action: AWS DeepRacer model details
Action Input: tr-reinvent-v1-clone
Could not obtain evaluation metrics 'Contents'

Observation: {'model_metadata_used_for_training': {'action_space_type': 'discrete', 'sensor': ['FRONT_FACING_CAMERA'], 'action_space': [{'steering_angle': -30, 'speed': 1.2, 'index': 0}, {'steering_angle': -30, 'speed': 1.4, 'index': 1}, {'steering_angle': -30, 'speed': 1.6, 'index': 2}, {'steering_angle': -20, 'speed': 1.8, 'index': 3}, {'steering_angle': -20, 'speed': 2, 'index': 4}, {'steering_angle': -20, 'speed': 2.2, 'index': 5}, {'steering_angle': -15, 'speed': 2.4, 'index': 6}, {'steering_angle': -15, 'speed': 2.6, 'index': 7}, {'steering_angle': -15, 'speed': 2.8, 'index': 8}, {'steering_angle': -10, 'speed': 3, 'index': 9}, {'steering_angle': -10, 'speed': 3.2, 'index': 10}, {'steering_a

In [45]:
question = "Should I change any of the hyperparameters that was used to train " + MODEL_NAME + " to improve the training?"
print_ww(react_agent( question)["output"])



> Entering new AgentExecutor chain...
 Here is the detailed response:

Question: Should I change any of the hyperparameters that was used to train tr-reinvent-v1-clone to improve the training?

Thought: To answer this question, I need more details on the tr-reinvent-v1-clone model and how it was trained.

Action: AWS DeepRacer model details
Action Input: tr-reinvent-v1-clone
Could not obtain evaluation metrics 'Contents'

Observation: {'model_metadata_used_for_training': {'action_space_type': 'discrete', 'sensor': ['FRONT_FACING_CAMERA'], 'action_space': [{'steering_angle': -30, 'speed': 1.2, 'index': 0}, {'steering_angle': -30, 'speed': 1.4, 'index': 1}, {'steering_angle': -30, 'speed': 1.6, 'index': 2}, {'steering_angle': -20, 'speed': 1.8, 'index': 3}, {'steering_angle': -20, 'speed': 2, 'index': 4}, {'steering_angle': -20, 'speed': 2.2, 'index': 5}, {'steering_angle': -15, 'speed': 2.4, 'index': 6}, {'steering_angle': -15, 'speed': 2.6, 'index': 7}, {'steering_angle': -15, 'speed

# Clean up

To avoid incurring charges let´s delete the downloaded models from S3

Upon completion of this step if you're not moving on to the stable diffusion workshop you should choose 'File', then 'Shutdown' and then choose 'Shutdown All' in the confirmation from the menu to stop incurring costs for the Sagemaker instance running this notebook.

In [ ]:
s3.delete_s3_prefix(DEEPRACER_EXPORT_S3_BUCKET, "")

# Recap

In this lab we integrated a Langchain agent with the AWS DeepRacer service using a custom tool. This allowed the LLM to help us with model analysis and how to improve them.


### Next steps:
[Amazon Bedrock workshop](https://github.com/aws-samples/amazon-bedrock-workshop) - More use cases and ways to build with Amazon Bedrock

[SageMaker JumpStart](https://docs.aws.amazon.com/sagemaker/latest/dg/studio-jumpstart.html) - Provides pre-trained, open-source models for a wide range of problem types to help you get started with machine learning.

